# Yield Sample Points

In [2]:
import geopandas as gpd

In [3]:
data = gpd.read_file('./gis/sampling_points.shp')

In [4]:
from wofost.wofost import InterfaceWOFOST
from paraallo.paraallo import InterfaceParaAllo
from apsim.apsim import InterfaceAPSIM
from map_module.yieldmapper import YieldMapGenerator

Platform not recognized, using system temp directory for PCSE settings.
Platform not recognized, using system temp directory for PCSE settings.


This notebook was built with:
python version: 3.9.16 (main, Mar  8 2023, 04:29:44) 
[Clang 14.0.6 ] 
PCSE version: 5.5.4


In [5]:
data

,id,lat,lon,geometry
0,1,18.752472,98.363832,POINT (98.36383 18.75247)
1,2,18.349639,100.230933,POINT (100.23093 18.34964)
2,3,16.715534,99.861514,POINT (99.86151 16.71553)
3,4,17.199650,102.283552,POINT (102.28355 17.19965)
4,5,17.177203,104.207918,POINT (104.20792 17.17720)
5,6,15.436608,104.255394,POINT (104.25539 15.43661)
6,7,14.296107,100.131246,POINT (100.13125 14.29611)
7,8,12.773269,101.902025,POINT (101.90202 12.77327)
8,9,8.367420,99.280452,POINT (99.28045 8.36742)
9,10,6.750630,101.067034,POINT (101.06703 6.75063)


# Run Crop Model for Sample points

In [8]:
# a function to generate all yield by latlon

import os
from tqdm import tqdm

def calculateCropYield(data, col_lat, col_lon):
    
    # Initializa Crop Models
    # Init WOFOST
    wofost = InterfaceWOFOST(os.getcwd())
    
    # Init APSIM
    apsim = InterfaceAPSIM(os.getcwd())
    
    # Init ParaRubberAllometry
    para_allo = InterfaceParaAllo()
    
    
    xmin, ymin, xmax, ymax = 97.3758964376, 5.69138418215, 105.589038527, 20.4178496363
    
    #crop_list = ['rice','maize','cassava', 'sugarcane', 'oilpalm', 'pararubber']
    
    crop_list = ['ri1','ri2', 'ri3', 'ri4', 'mp','cf','op', 'sc','rb']
    
    wofost_crop = ['ri1','ri2', 'ri3', 'ri4', 'mp','cf']
    
    apsim_crop = ['op', 'sc']
    
    allo_crop = ['rb']
    
    # Filter the data to only include points within the bounding box
    filtered_data = data[(data['lon'] >= xmin) & (data['lon'] <= xmax) & (data['lat'] >= ymin) & (data['lat'] <= ymax)]
    
    for crop in tqdm(crop_list):
        
        crop_yield_list = []
        
        for i in range(len(filtered_data)):

            #print(f'simulating {crop} yield {i+1}/{len(filtered_data)}')

            lat = filtered_data[col_lat][i]
            lon = filtered_data[col_lon][i]
            
            if crop in wofost_crop:
                
                crop_yield_list.append(wofost.get_sim_product(crop, lat, lon))
                
            elif crop in apsim_crop:
                
                crop_yield_list.append(apsim.get_sim_product(crop, lat, lon))
                
            else:
                
                crop_yield_list.append(para_allo.get_sim_product(crop, lat, lon))
        
        filtered_data[crop] = crop_yield_list
            
    return filtered_data
        
        
    
    
    

In [9]:
result_df = calculateCropYield(data, 'lat', 'lon')

100%|████████████████████████████████████████████████████████████| 9/9 [06:38<00:00, 44.28s/it]


In [7]:
result_df

,id,lat,lon,geometry,ri1,ri2,ri3,ri4,mp,cf,op,sc,rb
0,1,18.752472,98.363832,POINT (98.36383 18.75247),363,712,745,600,1630.951622,2492.770918,311.440872,5350.928,296.546319
1,2,18.349639,100.230933,POINT (100.23093 18.34964),363,712,745,600,1575.365535,2311.848100,995.886326,6291.680,280.130405
2,3,16.715534,99.861514,POINT (99.86151 16.71553),363,712,745,600,1318.457876,2055.786901,525.351830,5980.160,303.458484
3,4,17.199650,102.283552,POINT (102.28355 17.19965),363,712,745,600,1499.475480,2184.947239,555.945348,7027.520,291.857442
4,5,17.177203,104.207918,POINT (104.20792 17.17720),363,712,745,600,1615.307812,2206.672402,455.624293,6617.568,292.701512
5,6,15.436608,104.255394,POINT (104.25539 15.43661),363,712,745,600,1558.636198,2151.431438,688.219842,7162.560,310.285604
6,7,14.296107,100.131246,POINT (100.13125 14.29611),363,712,745,600,1427.103920,2142.390324,733.681762,6392.640,292.584635
7,8,12.773269,101.902025,POINT (101.90202 12.77327),363,712,745,600,1468.838725,2300.690860,923.896446,6919.296,307.928654
8,9,8.367420,99.280452,POINT (99.28045 8.36742),363,712,745,600,1241.128498,2363.073013,1392.713879,9074.512,309.753773
9,10,6.750630,101.067034,POINT (101.06703 6.75063),363,712,745,600,935.585696,2249.561708,1598.642258,12948.512,287.107485


# Turns Samples to Make Base Yield Maps

In [8]:
mapper = YieldMapGenerator(os.path.join(os.getcwd(),'./yield_maps'))

In [9]:
mapper.generateMaps(result_df)

generate ri1 map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/ri1.tif success
DONE!
generate ri2 map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/ri2.tif success
DONE!
generate ri3 map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/ri3.tif success
DONE!
generate ri4 map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/ri4.tif success
DONE!
generate mp map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/mp.tif success
DONE!
generate cf map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/CropModel3/CropModel3Production/./yield_maps/cf.tif success
DONE!
generate op map ...Make yield map at /Users/thantham/Documents/Outsource/P-Might/ABM/C

# Convert Agent Map to queriable data

In [2]:
import numpy as np
from array_compressor import compress_array, decompress_array

In [3]:
map_data = np.load('map_mockup.npy').astype('int16')
dtype = map_data.dtype
shape = map_data.shape
map_com = compress_array(map_data)

In [4]:
from agents_locator import convertAgentsMapToDataFrame

In [5]:
agents = convertAgentsMapToDataFrame(map_data)

100%|████████████████████████████████████████████████████████| 800/800 [00:26<00:00, 30.40it/s]


In [6]:
agents

,ids,lats,lons,areas
0,0,15.435877,99.790545,50
1,1,15.571046,99.771769,1
2,2,15.919986,100.482629,55
3,3,15.680135,100.035197,56
4,4,15.382618,100.227651,46
...,...,...,...,...
795,795,15.567005,99.758626,29
796,796,15.619714,100.421044,20
797,797,15.466731,100.125885,15
798,798,15.383169,99.150658,42


# Irrigation Label

In [7]:
import rasterio

def getIrrigation(data, irri_file):
    
    # make latlon set
    coordinateList = [(data['lons'][i], data['lats'][i]) for i in range(len(data))]
        
    with rasterio.open(irri_file) as src:
        
        # sampling yield value from baseyield raster
        data['irri'] =  [x[0] for x in src.sample(coordinateList)]
        
    return data

In [8]:
agents = getIrrigation(agents, './gis/irrigation_map.tif')

In [9]:
agents

,ids,lats,lons,areas,irri
0,0,15.435877,99.790545,50,1
1,1,15.571046,99.771769,1,0
2,2,15.919986,100.482629,55,0
3,3,15.680135,100.035197,56,0
4,4,15.382618,100.227651,46,1
...,...,...,...,...,...
795,795,15.567005,99.758626,29,0
796,796,15.619714,100.421044,20,0
797,797,15.466731,100.125885,15,0
798,798,15.383169,99.150658,42,0


# Let Agent Query Yield

In [10]:
from yield_module import CropModel

In [11]:
model = CropModel()

In [12]:
result = model.get_baseyield_1(agents)

In [13]:
result = model.vary_baseyield_2(result)

In [14]:
result = model.shock_temp_3(result, (15, 35))

In [15]:
result = model.shock_rain_4(result, (1800, 2200))

In [16]:
result = model.shock_disaster_5(result)

In [17]:
result

,ids,lats,lons,areas,irri,baseyld_ri1,baseyld_ri2,baseyld_ri3,baseyld_ri4,baseyld_mp,...,dis_ri1,dis_ri2,dis_ri3,dis_ri4,dis_mp,dis_cf,dis_sc,dis_op,dis_rb,dis_severity
0,0,15.435877,99.790545,50,1,363.0,712.0,745.0,600.0,1211.997025,...,299.660137,587.763134,615.004965,495.306012,909.477382,1737.173605,8224.232727,1436.213514,286.576652,0.000000
1,1,15.571046,99.771769,1,0,363.0,712.0,745.0,600.0,1204.057056,...,303.317054,594.935930,622.510208,501.350503,914.545405,1760.865098,8442.092343,1469.769784,290.168055,0.000000
2,2,15.919986,100.482629,55,0,363.0,712.0,745.0,600.0,1233.694933,...,300.897904,590.190930,617.545285,497.351907,929.583311,1778.659174,7866.944690,1295.590305,287.977025,0.000000
3,3,15.680135,100.035197,56,0,363.0,712.0,745.0,600.0,1219.894255,...,201.697411,395.615859,413.951987,333.384151,616.146357,1179.708729,5443.395165,929.162763,193.011251,0.744830
4,4,15.382618,100.227651,46,1,363.0,712.0,745.0,600.0,1250.849442,...,276.206489,541.760386,566.870067,456.539651,865.167689,1617.977410,7062.582233,1201.869150,264.414116,0.288117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,15.567005,99.758626,29,0,363.0,712.0,745.0,600.0,1202.629912,...,254.128622,498.456140,521.558742,420.047309,765.326865,1474.726811,7091.790479,1235.852781,243.110476,0.336337
796,796,15.619714,100.421044,20,0,363.0,712.0,745.0,600.0,1249.989745,...,307.852524,603.831947,631.818540,508.847147,963.630513,1814.573749,7856.307968,1312.026343,294.847740,0.000000
797,797,15.466731,100.125885,15,0,363.0,712.0,745.0,600.0,1238.578198,...,303.623828,595.537645,623.139811,501.857566,941.717535,1775.587563,7933.176602,1355.982148,290.577648,0.000000
798,798,15.383169,99.150658,42,0,363.0,712.0,745.0,600.0,1146.659494,...,263.177170,516.204256,540.129453,435.003586,755.690667,1498.989309,8107.172477,1457.494382,251.770249,0.408155


In [18]:
result = model.clean_agent_data(result, is_disaster=True)

In [19]:
result

,ids,areas,irri,ri1,ri2,ri3,ri4,mp,cf,sc,op,rb,severity
0,0,50,1,299.660137,587.763134,615.004965,495.306012,909.477382,1737.173605,8224.232727,1436.213514,286.576652,0.000000
1,1,1,0,303.317054,594.935930,622.510208,501.350503,914.545405,1760.865098,8442.092343,1469.769784,290.168055,0.000000
2,2,55,0,300.897904,590.190930,617.545285,497.351907,929.583311,1778.659174,7866.944690,1295.590305,287.977025,0.000000
3,3,56,0,201.697411,395.615859,413.951987,333.384151,616.146357,1179.708729,5443.395165,929.162763,193.011251,0.744830
4,4,46,1,276.206489,541.760386,566.870067,456.539651,865.167689,1617.977410,7062.582233,1201.869150,264.414116,0.288117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,29,0,254.128622,498.456140,521.558742,420.047309,765.326865,1474.726811,7091.790479,1235.852781,243.110476,0.336337
796,796,20,0,307.852524,603.831947,631.818540,508.847147,963.630513,1814.573749,7856.307968,1312.026343,294.847740,0.000000
797,797,15,0,303.623828,595.537645,623.139811,501.857566,941.717535,1775.587563,7933.176602,1355.982148,290.577648,0.000000
798,798,42,0,263.177170,516.204256,540.129453,435.003586,755.690667,1498.989309,8107.172477,1457.494382,251.770249,0.408155


# Calculate Agent's Production

In [20]:
result

,ids,areas,irri,ri1,ri2,ri3,ri4,mp,cf,sc,op,rb,severity
0,0,50,1,299.660137,587.763134,615.004965,495.306012,909.477382,1737.173605,8224.232727,1436.213514,286.576652,0.000000
1,1,1,0,303.317054,594.935930,622.510208,501.350503,914.545405,1760.865098,8442.092343,1469.769784,290.168055,0.000000
2,2,55,0,300.897904,590.190930,617.545285,497.351907,929.583311,1778.659174,7866.944690,1295.590305,287.977025,0.000000
3,3,56,0,201.697411,395.615859,413.951987,333.384151,616.146357,1179.708729,5443.395165,929.162763,193.011251,0.744830
4,4,46,1,276.206489,541.760386,566.870067,456.539651,865.167689,1617.977410,7062.582233,1201.869150,264.414116,0.288117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,29,0,254.128622,498.456140,521.558742,420.047309,765.326865,1474.726811,7091.790479,1235.852781,243.110476,0.336337
796,796,20,0,307.852524,603.831947,631.818540,508.847147,963.630513,1814.573749,7856.307968,1312.026343,294.847740,0.000000
797,797,15,0,303.623828,595.537645,623.139811,501.857566,941.717535,1775.587563,7933.176602,1355.982148,290.577648,0.000000
798,798,42,0,263.177170,516.204256,540.129453,435.003586,755.690667,1498.989309,8107.172477,1457.494382,251.770249,0.408155


In [4]:
# a function to convert grid data to agent map

In [21]:
def calculateCropProduct(data, crop_list, area_col):
    
    area_array = np.asarray(data[area_col])
    
    for crop in crop_list:
                                    
        data[crop] = data[crop] * data[area_col]
        
    return data

In [22]:
data_prod = calculateCropProduct(result, ['ri1','ri2', 'ri3', 'ri4', 'mp','cf','op', 'sc','rb'], 'areas')

/var/folders/d1/pv_vgtx525d9v64kb01fvbz80000gn/T/ipykernel_91595/1700776610.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[crop] = data[crop] * data[area_col]


In [23]:
data_prod

,ids,areas,irri,ri1,ri2,ri3,ri4,mp,cf,sc,op,rb,severity
0,0,50,1,14983.006860,29388.156705,30750.248238,24765.300594,45473.869081,86858.680275,411211.636364,71810.675692,14328.832580,0.000000
1,1,1,0,303.317054,594.935930,622.510208,501.350503,914.545405,1760.865098,8442.092343,1469.769784,290.168055,0.000000
2,2,55,0,16549.384709,32460.501137,33964.990656,27354.354891,51127.082114,97826.254577,432681.957952,71257.466773,15838.736389,0.000000
3,3,56,0,11295.055025,22154.488093,23181.311277,18669.512438,34504.195983,66063.688850,304830.129266,52033.114733,10808.630047,0.744830
4,4,46,1,12705.498496,24920.977766,26076.023084,21000.823961,39797.713701,74426.960873,324878.782703,55285.980922,12163.049317,0.288117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,29,0,7369.730041,14455.228069,15125.203527,12181.371968,22194.479071,42767.077521,205661.923882,35839.730655,7050.203808,0.336337
796,796,20,0,6157.050474,12076.638946,12636.370807,10176.942932,19272.610257,36291.474979,157126.159359,26240.526857,5896.954796,0.000000
797,797,15,0,4554.357413,8933.064677,9347.097169,7527.863492,14125.763018,26633.813442,118997.649031,20339.732219,4358.664720,0.000000
798,798,42,0,11053.441131,21680.578747,22685.437031,18270.150629,31739.008011,62957.550970,340501.244048,61214.764054,10574.350454,0.408155


In [37]:
import pandas as pd
import json

In [38]:
data_json = data_prod.to_json()

In [41]:
pd.DataFrame(json.loads(data_json))

,ids,areas,irri,ri1,ri2,ri3,ri4,mp,cf,sc,op,rb,severity
0,0,50,1,14983.006860,29388.156705,30750.248238,24765.300594,45473.869081,86858.680275,411211.636364,71810.675692,14328.832580,0.000000
1,1,1,0,303.317054,594.935930,622.510208,501.350503,914.545405,1760.865098,8442.092343,1469.769784,290.168055,0.000000
2,2,55,0,16549.384709,32460.501137,33964.990656,27354.354891,51127.082114,97826.254577,432681.957952,71257.466773,15838.736389,0.000000
3,3,56,0,11295.055025,22154.488093,23181.311277,18669.512438,34504.195983,66063.688850,304830.129266,52033.114733,10808.630047,0.744830
4,4,46,1,12705.498496,24920.977766,26076.023084,21000.823961,39797.713701,74426.960873,324878.782703,55285.980922,12163.049317,0.288117
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,795,29,0,7369.730041,14455.228069,15125.203527,12181.371968,22194.479071,42767.077521,205661.923882,35839.730655,7050.203808,0.336337
796,796,20,0,6157.050474,12076.638946,12636.370807,10176.942932,19272.610257,36291.474979,157126.159359,26240.526857,5896.954796,0.000000
797,797,15,0,4554.357413,8933.064677,9347.097169,7527.863492,14125.763018,26633.813442,118997.649031,20339.732219,4358.664720,0.000000
798,798,42,0,11053.441131,21680.578747,22685.437031,18270.150629,31739.008011,62957.550970,340501.244048,61214.764054,10574.350454,0.408155
